In [3]:
import os, uuid, sys
from azure.storage.filedatalake import DataLakeServiceClient
from azure.storage.filedatalake import FileSystemClient
from azure.core._match_conditions import MatchConditions
from azure.storage.filedatalake._models import ContentSettings
import pandas as pd


In [4]:
# Connection String to Datalake 
connectionstring="DefaultEndpointsProtocol=https;AccountName=datalaketest123;AccountKey=mG5IGAwpT+tjzf7XQ7TDKqsrpN2ESnMG/CDjPUjSIcUn46mKKlF67b1mRBXDq/J59NuNRImYw/qgCUuTFFX2dg==;EndpointSuffix=core.windows.net"

In [5]:
# The datalake service client is the class that interacts on account level with the datalake: Goal list the different filesystems/blobs
service_client = DataLakeServiceClient.from_connection_string(connectionstring)
file_systems = service_client.list_file_systems()

file_systems_list =[]
for file_system in file_systems:
    file_systems_list.append(file_system.name)
print(file_systems_list)

In [7]:
# The FileSystemClient class interacts with a specific filesystem in the datalake: Goal get the pathnames of our desired filesystem
for file_system in file_systems_list:
    paths = FileSystemClient.from_connection_string(connectionstring, file_system_name=file_system).get_paths()
    for path in paths:
        print(path.name + '\n')

d365datalakeexport

d365datalakeexport/ChangeFeed

d365datalakeexport/Tables

d365datalakeexport/Tables/Common

d365datalakeexport/Tables/Common/Customer

d365datalakeexport/Tables/Common/Customer/Main

d365datalakeexport/Tables/Common/Customer/Main/CustTable

d365datalakeexport/Tables/Common/Customer/Main/CustTable/CUSTTABLE_00001.csv

d365datalakeexport/Tables/Common/Customer/Main/CustTable/index.json

d365datalakeexport/Tables/Common/Customer/Main/CustTable.cdm.json

d365datalakeexport/Tables/SupplyChain

d365datalakeexport/Tables/SupplyChain/ProductInformationManagement

d365datalakeexport/Tables/SupplyChain/ProductInformationManagement/Main

d365datalakeexport/Tables/SupplyChain/ProductInformationManagement/Main/EcoResProduct

d365datalakeexport/Tables/SupplyChain/ProductInformationManagement/Main/EcoResProduct/ECORESPRODUCT_00001.csv

d365datalakeexport/Tables/SupplyChain/ProductInformationManagement/Main/EcoResProduct/index.json

d365datalakeexport/Tables/SupplyChain/ProductInfo

In [89]:
# Create a dataframe with the desired files. The csv files contain the data, the cdm.json files the metadata
import re
pattern = re.compile('\w.*\.cdm.json|\w.*\.csv')
results = [(path.name, 
path.name.split('/')[-1],
path.name.split('/')[-1].split('.')[0].split('_')[0].lower(),
path.name.split('/')[-1].split('.')[-1]) for path in FileSystemClient.from_connection_string(connectionstring, file_system_name=file_system).get_paths() \
    if pattern.match(path.name)]
# pattern = re.compile('\w.*\.cdm.json|\w.*\.csv')
# new_results = list(filter(pattern.match,results))
pd.DataFrame(results, columns=['Path','Filename', 'Table', 'Type'])

,Path,Filename,Table,Type
0,d365datalakeexport/Tables/Common/Customer/Main...,CUSTTABLE_00001.csv,custtable,csv
1,d365datalakeexport/Tables/Common/Customer/Main...,CustTable.cdm.json,custtable,json
2,d365datalakeexport/Tables/SupplyChain/ProductI...,ECORESPRODUCT_00001.csv,ecoresproduct,csv
3,d365datalakeexport/Tables/SupplyChain/ProductI...,EcoResProduct.cdm.json,ecoresproduct,json


In [253]:
# Read file into a dataframe
from io import StringIO
adl = FileSystemClient.from_connection_string(connectionstring, file_system_name=file_system)
input_file = adl.get_file_client("d365datalakeexport/Tables/Common/Customer/Main/CustTable/CUSTTABLE_00001.csv")

customers = pd.read_csv(StringIO(re.sub('"""', '"', str(input_file.download_file().readall(),'utf-8'))),sep=';', dayfirst=True, infer_datetime_format=True, header=None)

In [255]:
# Read metadata json file for columnames
import json
adl = FileSystemClient.from_connection_string(connectionstring, file_system_name=file_system)
metadata_file =adl.get_file_client("d365datalakeexport/Tables/Common/Customer/Main/CustTable.cdm.json")
json_test = json.loads(metadata_file.download_file().readall())

columns_names = [item['name'] for item in json_test['definitions'][0]['hasAttributes']]
customers.columns = columns_names
customers
#pd.json_normalize(json_test, record_path=['definitions','hasAttributes'])



,_SysRowId,LSN,LastProcessedChange_DateTime,DataLakeModified_DateTime,RECID,PAYMTERMID,LINEDISC,ACCOUNTNUM,ACCOUNTSTATEMENT,BANKACCOUNT,...,USEPURCHREQUEST,VATNUM,VENDACCOUNT,WEBSALESORDERDISPLAY,FOREIGNERID_BR,BLOCKFLOORLIMITUSEINCHANNEL,CREATEDDATETIME,MODIFIEDDATETIME,DATAAREAID,ROWVERSION
0,1,1,5.11.2021 13:34,44505.56389,22565431405,Net10,NaN,4003,0,NaN,...,0,NaN,NaN,0,NaN,0,2017-01-28 15:10:08.000,2017-01-28 17:22:30.000,usrt,0x0000000001D3F466
1,2,2,5.11.2021 13:35,44506.56389,22565431406,Net10,NaN,4005,0,NaN,...,0,NaN,NaN,0,NaN,0,2017-01-28 15:18:49.000,2017-01-28 15:21:17.000,usrt,0x0000000001D3F467
2,3,3,5.11.2021 13:36,44507.56389,22565431407,Net10,NaN,4007,0,NaN,...,0,NaN,NaN,0,NaN,0,2017-01-28 15:22:02.000,2017-01-28 15:23:33.000,usrt,0x0000000001D3F468
3,4,4,5.11.2021 13:37,44508.56389,22565431408,Net10,NaN,4009,0,NaN,...,0,NaN,NaN,0,NaN,0,2017-01-28 15:29:51.000,2017-01-28 15:31:26.000,usrt,0x0000000001D3F469
4,5,5,5.11.2021 13:38,44509.56389,22565431409,Net10,NaN,4011,0,NaN,...,0,NaN,NaN,0,NaN,0,2017-01-28 15:32:09.000,2017-04-08 13:49:14.000,usrt,0x0000000001D3F46A
5,6,6,5.11.2021 13:39,44510.56389,22565431410,Net10,NaN,4013,0,NaN,...,0,NaN,NaN,0,NaN,0,2017-01-28 15:34:23.000,2017-01-28 15:36:12.000,usrt,0x0000000001D3F46B
6,7,7,5.11.2021 13:40,44511.56389,22565431411,Net10,NaN,4015,0,NaN,...,0,NaN,NaN,0,NaN,0,2017-01-28 15:56:51.000,2017-01-28 16:50:58.000,usrt,0x0000000001D3F46C
7,8,8,5.11.2021 13:41,44512.56389,22565431412,Net10,NaN,4017,0,NaN,...,0,NaN,NaN,0,NaN,0,2017-01-28 17:17:10.000,2017-01-28 17:21:36.000,usrt,0x0000000001D3F46D
8,9,9,5.11.2021 13:42,44513.56389,22565428385,Net 10,NaN,100001,0,NaN,...,0,NaN,NaN,0,NaN,0,2017-02-14 23:06:26.000,2018-02-21 18:26:05.000,frrt,0x0000000001D3F46E
9,10,10,5.11.2021 13:43,44514.56389,22565422808,Net10,NaN,100001,0,NaN,...,0,NaN,NaN,0,NaN,0,2016-10-06 20:03:04.000,2018-02-21 18:20:35.000,usrt,0x0000000001D3F46F
